Source: https://github.com/UKPLab/sentence-transformers

In [7]:
!pip install pycarol

In [ ]:
!pip install -U sentence-transformers

In [13]:
from pycarol import Carol, Staging, ApiKeyAuth, Storage, PwdAuth
import pandas as pd
import pickle
import numpy as np
import re
import math

In [14]:
login_dont = Carol(domain='monitoriaqa',
app_name=' ',
organization='totvs',
auth=PwdAuth('fmendes@totvs.com.br', 'Intxdx70248821*'))

In [15]:
df = Staging(login_dont).fetch_parquet(staging_name='articles',
                        connector_name='carol_connect_zerado',
                        cds=True
                        )

100%|██████████| 157/157 [03:27<00:00,  1.32s/it]


In [16]:
df = df.astype(str)

In [17]:
df.shape

(37290, 26)

In [18]:
df = df[df['title'].str.contains('CST - LGPD')]

In [19]:
df.shape

(53, 26)

In [32]:
df.iloc[45].body

'<p><strong>Dúvida</strong><strong><br></strong><span data-sheets-value=\'{"1":2,"2":"Quais os canais disponíveis para o cliente se comunicar com a TOTVS em assuntos relacionados à LGPD?"}\' data-sheets-userformat=\'{"2":949,"3":{"1":0},"5":{"1":[{"1":2,"2":0,"5":{"1":2,"2":0}},{"1":0,"2":0,"3":3},{"1":1,"2":0,"4":2}]},"7":{"1":[{"1":2,"2":0,"5":{"1":2,"2":0}},{"1":0,"2":0,"3":3},{"1":1,"2":0,"4":1}]},"8":{"1":[{"1":2,"2":0,"5":{"1":2,"2":0}},{"1":0,"2":0,"3":3},{"1":1,"2":0,"4":1}]},"10":1,"11":4,"12":0}\'>Qual o objetivo do canal de atendimento para direitos dos titulares?</span><br> <br> <strong>Ambiente</strong><strong><br></strong>Centro de Serviço - Lei Geral de Proteção de Dados - Todos<br><br><strong>Solução</strong><strong><br></strong>O canal de atendimento para direitos dos titulares, visa demonstrar o compromisso que a TOTVS possui com acesso aos dados pessoais seja de clientes, fornecedores, funcionários, ex-funcionários, visitantes, entre outros.<br><br>O titular pode a q

In [33]:
df.title.unique()

array(['CST - LGPD - Atualização de dados de Parceiros',
       'CST - LGPD - Acesso aos dados de fornecedores inativos',
       'CST - LGPD - Solicitar exclusão, anonimização e/ou bloqueio de dados de Fornecedor Inativo',
       'CST - LGPD - Tempo de armazenamento de dados posterior ao cancelamento de contrato',
       'CST - LGPD - Anonimização, bloqueio ou exclusão dados Parceiro',
       'CST - LGPD - Acesso aos meus dados pessoais  cadastrados na TOTVS',
       'CST - LGPD - Armazenamento de dados após desligamento',
       'CST - LGPD - Dados de candidatura das vagas na TOTVS',
       'CST - LGPD - Bloqueio de dados com processo judicial em andamento',
       'CST - LGPD - Compartilhamento de dados pessoais',
       'CST - LGPD - Acesso aos dados pessoais cadastrados na TOTVS',
       'CST - LGPD - Solicitar anonimização por funcionário ativo',
       'CST - LGPD - Utilização de dados pessoais dos dependentes de funcionários',
       'CST - LGPD - Tratativas de dados pessoais de

In [34]:
df = df.replace('', np.nan)

In [35]:
df = df[(df.source_locale == 'pt-br') & (~df.body.isnull())]

In [36]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    return " ".join(re.sub(r'\s([?.!"](?:\s|$))', r'\1', text).strip().split())

In [37]:
def get_question(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>D(ú|u)vida).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
        
    m = re.search('(?<=<strong>Ocorr(ê|e)ncia).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
    
    return np.nan

In [38]:
def get_question_type(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>D(ú|u)vida).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return 'question'

    m = re.search('(?<=<strong>Ocorr(ê|e)ncia).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return 'occurrence'
    
    return np.nan

In [39]:
def get_environment(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?=<strong>Ambiente).*?(?=<strong>Solu(ç|c)(ã|a)o)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return remove_html_tags(m.group(0))

In [40]:
def get_solution(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>Solu(ç|c)(ã|a)o)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return m.group(0)

In [41]:
def get_sanitized_solution(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>Solu(ç|c)(ã|a)o)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return re.sub('\.(?!\s)(?!$)(?!com)', '. ', remove_html_tags(m.group(0)))

In [42]:
df['question'] = df.body.apply(get_question)
df['question_type'] = df.body.apply(get_question_type)
df['environment'] = df.body.apply(get_environment)
df['solution'] = df.body.apply(get_solution)
df['sanitized_solution'] = df.body.apply(get_sanitized_solution)

In [43]:
df = df.replace('', np.nan)

In [44]:
df.head()

,__v,author_id,body,brand_id,brand_name,comments_disabled,created_at,draft,edited_at,html_url,...,updated,updated_at,url,vote_count,vote_sum,question,question_type,environment,solution,sanitized_solution
4502,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,True,2020-06-09T17:56:30.000Z,True,2020-06-09T17:56:30.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-06-09T17:56:30.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Tenho cadastro na TOTVS e gostaria de atualiza...,question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong>Como a TOTVS apenas ...,Como a TOTVS apenas trabalha com Parceiros sen...
4504,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-06-09T17:52:01.000Z,True,2020-06-09T17:52:01.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-06-09T17:52:01.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Como posso obter acesso aos meus dados pessoai...,question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong>Será necessário aber...,Será necessário abertura do ticket para o time...
4507,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-06-09T17:47:59.000Z,True,2020-06-09T17:47:59.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-06-09T17:47:59.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,"Como posso solicitar exclusão, anonimização e/...",question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong><span data-sheets-va...,"Não será possível solicitar exclusão, anonimiz..."
4508,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-06-09T17:45:45.000Z,True,2020-06-09T17:48:37.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-06-09T17:48:37.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Por quanto tempo a TOTVS armazena os dados pes...,question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong>Os dados utilizados ...,Os dados utilizados para cadastro continuam se...
4540,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-06-09T19:25:09.000Z,True,2020-06-09T19:25:09.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-06-09T19:25:09.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,"Posso solicitar anonimização, bloqueio ou excl...",question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong><span data-sheets-va...,"Não será possível solicitar anonimização, bloq..."


In [45]:
df = df.dropna(subset=['question', 'question_type', 'environment', 'solution'])

In [46]:
df = df.reset_index(drop=True)

In [47]:
df['question'].count()

53

In [48]:
df.iloc[5].sanitized_solution

'Para verificar o acesso aos seus dados pessoais, você pode entrar em contato conosco através deste Portal, no site da totvs.com para saber mais sobre os seus dados.'

In [49]:
df['id'] = df['id'].apply(lambda x: x.replace('.0', ''))

In [50]:
df.iloc[0].id

'360049262914'

In [51]:
target = Carol(domain='mendes',
app_name='searchlgpd',
organization='datascience',
auth=ApiKeyAuth('bf56760357424b60ae9558bcf9f090c2'),
connector_id='175e781f1cce470eb0457323430f1407')

In [52]:
target_stag = Staging(target)

In [53]:
target_stag.send_data(staging_name='lgpd_articles', connector_id='175e781f1cce470eb0457323430f1407', data=df,)
#                             crosswalk_auto_create=['idinternalmdb'], auto_create_schema=True)

fetched crosswalk  ['idinternalmdb']


### Get sentence embeddings

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [30]:
# Embed a list of sentences
sentence_embeddings = model.encode(df.question.values)

In [32]:
len(sentence_embeddings)

14

In [33]:
id_list = df['id'].values

In [34]:
len(id_list)

14

In [35]:
index_to_question_id_mapping = dict(enumerate(id_list))
question_id_to_index_mapping = {j: i for i,j in index_to_question_id_mapping.items()} 

### Save objects in Storage

In [36]:
stg = Storage(target)

In [37]:
def save_object_to_storage(obj, filename):
    with open(filename, "bw") as f:
        pickle.dump(obj, f)
    stg.save(filename, obj, format='pickle')

In [38]:
save_object_to_storage(sentence_embeddings, 'sentence_embeddings')

In [39]:
save_object_to_storage(index_to_question_id_mapping, 'index_to_question_id_mapping')

In [40]:
save_object_to_storage(question_id_to_index_mapping, 'question_id_to_index_mapping')

In [1]:
!python --version

Python 3.8.2


In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler

In [2]:
# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [21]:
import pickle

In [5]:
with open('encoder', "bw") as f:
    pickle.dump(model, f)

/Users/felipe/opt/miniconda3/envs/recommendation/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [22]:
sentence_embeddings = pickle.load(open("sentence_embeddings", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'sentence_embeddings'

In [23]:
sentence_embeddings[:10]

NameError: name 'sentence_embeddings' is not defined

In [ ]:
df.index

In [28]:
df.iloc[12].question

"Após configurar HTTPS no fluig, ao acessá-lo pelo aplicativo fluig Mobile e informar servidor de conexão é apresentado o erro 'Falha na comunicação'."

In [144]:
def get_most_similar_question(query):

    query_vec = model.encode([query])
    k = 5

    # compute normalized dot product as score
    score = np.sum(query_vec[0] * sentence_embeddings, axis=1) / np.linalg.norm(sentence_embeddings, axis=1)
    topk_scores = np.sort(score)[::-1]
    topk_idx = np.argsort(score)[::-1]
    return topk_idx[:k], topk_scores[:k]

In [119]:
#size = np.int32(np.floor(0.2 * questions.shape[0]))
size = 100

indices = np.random.choice(
    df.question.index, 
    replace=False,
    size=size
).tolist()

In [71]:
questions = []

In [93]:
df.iloc[indices[9]].question

'Solicito uma função para conversão de um arquivo .doc para .PDF'

In [94]:
questions.append('função para converter arquivo doc para pdf')

In [97]:
len(questions)

10

In [145]:
get_most_similar_question('adicionar transportadora nas notas fiscais geradas na rotina 1322')

(array([ 4270, 23653, 14174, 26242, 27618]),
 array([0.64584833, 0.60263   , 0.5729962 , 0.54728776, 0.54643303],
       dtype=float32))

In [150]:
df.iloc[4270].question

'Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.'

In [126]:
for i, query in enumerate(questions):
    raw_query = df.iloc[indices[i]].question
    most_similar = get_most_similar_question(query)
    if most_similar != i:
        print(f'{query}')
        print(f'{indices[i]} - {raw_query}')
        print(f'{most_similar} - {df.iloc[most_similar].question}')

0.645848274230957 - 0.645848274230957
0.6026300191879272 - 0.6026300191879272
0.5729961395263672 - 0.5729961395263672
0.5472877025604248 - 0.5472877025604248
0.5464330315589905 - 0.5464330315589905
0.5448733568191528 - 0.5448733568191528
0.5358591079711914 - 0.5358591079711914
0.531542956829071 - 0.531542956829071
0.5271217823028564 - 0.5271217823028564
0.5169530510902405 - 0.5169530510902405
adicionar transportadora nas notas fiscais geradas na rotina 1322
17835 - Por que um determinado lançamento não é apresentado na rotina 723?
4270 - Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.
0.5572085976600647 - 0.5572085976600647
0.5351568460464478 - 0.5351568460464478
0.5280529856681824 - 0.5280529856681824
0.5269234776496887 - 0.5269234776496887
0.5140586495399475 - 0.5140586495399475
0.5049834847450256 - 0.5049834847450256
0.49780139327049255 - 0.49780139327049255
0.4954758882522583 - 0.4954758882522583
0.49390849471092224 - 0.4939

In [115]:
df.iloc[23587].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360035902474-LG-PAT-Formas-de-transfer%C3%AAncia-de-um-bem-para-outro-na-mesma-empresa'

In [114]:
df.iloc[17132].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360025943751-LG-PAT-Como-efetuar-a-transfer%C3%AAncia-de-bens-entre-empresas'

In [112]:
df.iloc[1527].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360045420414-WINT-Como-utilizar-a-rotina-2039-Cadastro-de-Finalizadora'

In [32]:
from pycarol import Query

In [33]:
from pycarol.filter import Filter, TYPE_FILTER, TERMS_FILTER

In [30]:
def get_similar_questions(model, sentence_embeddings, query, k):
    query_vec = model.encode([query])
    score = np.sum(query_vec[0] * sentence_embeddings, axis=1) / np.linalg.norm(sentence_embeddings, axis=1)
    print(score[:100])
    topk_scores = np.sort(score)[::-1]
    topk_idx = np.argsort(score)[::-1]
    print(topk_idx[:k], topk_scores[:k])
    return list(topk_idx[:k]), list(topk_scores[:k])

In [31]:
def get_questions_by_ids(question_ids):
    dm_name = 'articles'
    query_filter = Filter.Builder().must(TYPE_FILTER(value=f'{dm_name}Golden')) \
        .must(TERMS_FILTER(key='mdmGoldenFieldAndValues.id.raw', value=question_ids)).build().to_json()
    page_size=1000
    return Query(target, page_size=page_size).query(query_filter).go().results

In [34]:
def query(query):
    k = 5
    topk_idx, topk_scores = get_similar_questions(model, sentence_embeddings, query, k)
    question_ids = [index_to_question_id_mapping[idx] for idx in topk_idx]
    results = get_questions_by_ids(question_ids)
    response = ''
    for i, question_id in enumerate(question_ids):
        result = [result for result in results if result['id'] == question_id][0]
        response = response + 'Pergunta similar: ' + result['question'] + '\n'
        response = response + 'Resposta: ' + result['solution'] + '\n'
        response = response + 'Score: ' + str(topk_scores[topk_idx.index(question_id_to_index_mapping[question_id])])
        if i != 4:
            response = response +  '\n\n'
    # for i, idx in enumerate(topk_idx):
    #     question_id = index_to_question_id_mapping[idx]
    #     question = get_question_by_id(question_id)[0]
    #     response = response + 'Pergunta similar: ' + question.question + '\n'
    #     response = response + 'Resposta: ' + question.solution + '\n'
    #     response = response + 'Score: ' + topk_scores[i]
    #     if i != 4:
    #         response = response +  '\n\n'
    return {'response': response}

In [35]:
query('adicionar transportadora nas notas fiscais geradas na rotina 1322')

[ 0.02887835 -0.00526772 -0.02707283 -0.03857228  0.03806132  0.04284458
  0.0201705   0.06050855  0.05173742  0.03805823  0.02416999  0.04116369
 -0.06460869  0.09727325  0.08634991  0.01915933  0.08498693 -0.05493666
  0.02492339 -0.00886754  0.10098135  0.06362017  0.0776341  -0.02791644
  0.01163275  0.0192889  -0.01115312 -0.02294862 -0.00829241 -0.03112706
  0.0017001  -0.04119982 -0.01010577 -0.00929159  0.05805634  0.00259353
 -0.02442231 -0.040168    0.00310044 -0.05039007 -0.00759398 -0.03024388
 -0.05424359  0.11716861  0.05606491  0.02613153  0.07075207 -0.02040705
  0.00579199  0.05455872 -0.05031964  0.01694646  0.04972199  0.01136606
  0.01005764 -0.02961715  0.05143109  0.02394645 -0.05122325 -0.06841542
 -0.0735998  -0.02130548  0.04605877 -0.08823569 -0.0147755   0.09354758
  0.02888806 -0.02228448  0.09114084  0.04604091  0.07873157 -0.05889153
  0.09564672  0.12373262 -0.00022965 -0.04361885  0.00578425 -0.03231039
  0.00238829  0.0007079   0.11109977 -0.00406435  0

NameError: name 'index_to_question_id_mapping' is not defined

In [4]:
import requests

In [11]:
query = 'adicionar transportadora nas notas fiscais geradas na rotina 1322'

In [9]:
query = 'qual a politica da LGPD?'

In [12]:
data = {
    'query': query,
    'k': '3',
    'threshold': '65'
}

response = requests.post(url='https://mendes-searchlgpd.apps.carol.ai/query',
                                 json=data)
print(response)
print(response.text)
response.json()

<Response [200]>
{"response":[],"session_id":1}



{'response': [], 'session_id': 1}